In [1]:
from django_bootstrap import bootstrap
bootstrap()

from django.core import serializers
import json

In [2]:
from criteria.models import *

In [3]:
project_ids = [29,30,31]

In [4]:
criterias = {}

for cupt_code in MajorCuptCode.objects.all():
    data1 = serializers.serialize('json',[cupt_code])
    data2 = json.loads(data1)[0]['fields']
    
    criterias[(cupt_code.program_code,cupt_code.major_code)] = {
        'data': {
            'major_cupt_code': data2,
            'criterias': {},
        },
        'objects': {
            'cupt_code': cupt_code,
            'criterias': {},
        }
    }
    
print(criterias)

{('10020101111801A', ''): {'data': {'major_cupt_code': {'program_code': '10020101111801A', 'program_type': 'ภาษาไทย ปกติ', 'program_type_code': 'A', 'faculty': 2, 'major_code': '', 'title': 'วท.บ. สาขาวิชาอาหาร โภชนาการ และการกำหนดอาหาร', 'major_title': '', 'component_weight_type': '11'}, 'criterias': {}}, 'objects': {'cupt_code': <MajorCuptCode: วท.บ. สาขาวิชาอาหาร โภชนาการ และการกำหนดอาหาร (ภาษาไทย ปกติ)>, 'criterias': {}}}, ('10020101500101A', ''): {'data': {'major_cupt_code': {'program_code': '10020101500101A', 'program_type': 'ภาษาไทย ปกติ', 'program_type_code': 'A', 'faculty': 2, 'major_code': '', 'title': 'วท.บ. สาขาวิชาเทคโนโลยีระบบเกษตร', 'major_title': '', 'component_weight_type': '50'}, 'criterias': {}}, 'objects': {'cupt_code': <MajorCuptCode: วท.บ. สาขาวิชาเทคโนโลยีระบบเกษตร (ภาษาไทย ปกติ)>, 'criterias': {}}}, ('10020101500201A', ''): {'data': {'major_cupt_code': {'program_code': '10020101500201A', 'program_type': 'ภาษาไทย ปกติ', 'program_type_code': 'A', 'faculty': 2, 'ma

In [5]:
cupt_code

<MajorCuptCode: วศ.บ. สาขาวิชาวิศวกรรมการบินและอวกาศ (Double Degree - นานาชาติ)>

In [6]:
def cb(d1,d2):
    res = {}
    for k in d1:
        res[k] = d1[k]
    for k in d2:
        res[k] = d2[k]
    return res

def serialize_admission_criteria(criteria):
    data1 = serializers.serialize('json',[criteria])
    data2 = json.loads(data1)[0]
    
    scores = list(criteria.scorecriteria_set.all())
    score_data1 = serializers.serialize('json', scores)
    score_data2 = json.loads(score_data1)
    
    result = {
        'admission_criteria': data2['fields'],
        'score_criterias': [cb(d['fields'],{'pk': d['pk']}) for d in score_data2],
    }

    #print('data',score_data2)
    
    print(result)
    return result

In [7]:
#c = 0

for p,m in criterias:
    k = (p,m)
    cupt_code = criterias[k]['objects']['cupt_code']
    print(cupt_code)
    curriculum_majors = CurriculumMajor.objects.filter(cupt_code=cupt_code)
    for m in curriculum_majors:
        if m.admission_project_id not in project_ids:
            continue

        print(m.admission_project_id)
        major_admission_criterias = CurriculumMajorAdmissionCriteria.objects.filter(curriculum_major=m)
        
        criterias[k]['objects']['criterias'][m.admission_project_id] = [
            {'slots':cmac.slots,
             'admission_criteria': cmac.admission_criteria}
            for cmac in major_admission_criterias
            if cmac.admission_criteria.is_deleted == False
        ]
        criterias[k]['data']['criterias'][m.admission_project_id] = [
            {'slots':cmac.slots,
             'admission_criteria': serialize_admission_criteria(cmac.admission_criteria)}
            for cmac in major_admission_criterias
            if cmac.admission_criteria.is_deleted == False
        ]
        
    print('------------------')
    print()

วท.บ. สาขาวิชาอาหาร โภชนาการ และการกำหนดอาหาร (ภาษาไทย ปกติ)
------------------

วท.บ. สาขาวิชาเทคโนโลยีระบบเกษตร (ภาษาไทย ปกติ)
------------------

วท.บ. สาขาวิชาการจัดการศัตรูพืชและสัตว์ (ภาษาไทย ปกติ)
31
{'admission_criteria': {'admission_project': 31, 'faculty': 2, 'version': 3, 'is_deleted': False, 'created_at': '2021-06-04T15:08:13.677', 'created_by': 'regis', 'additional_description': '', 'additional_condition': '', 'curriculum_majors_json': '[{"curriculum_major_id": 2079, "slots": 3, "major_cupt_code_id": 80, "program_code": "10020101500201A", "program_type_code": "A", "major_code": ""}]'}, 'score_criterias': [{'admission_criteria': 7986, 'version': 3, 'primary_order': 1, 'secondary_order': 0, 'criteria_type': 'required', 'score_type': 'OTHER', 'value': '2.5000', 'unit': 'คะแนน', 'description': 'กำลังศึกษาหรือสำเร็จศึกษาชั้นมัธยมศึกษาปีที่ 6 หรือเทียบเท่า', 'parent': None, 'relation': None, 'pk': 70111}, {'admission_criteria': 7986, 'version': 3, 'primary_order': 1, 'secondary_

{'admission_criteria': {'admission_project': 31, 'faculty': 34, 'version': 2, 'is_deleted': False, 'created_at': '2021-06-04T14:33:55.656', 'created_by': 'cond34', 'additional_description': '', 'additional_condition': '', 'curriculum_majors_json': '[{"curriculum_major_id": 2100, "slots": 45, "major_cupt_code_id": 52, "program_code": "10020215210101A", "program_type_code": "A", "major_code": ""}]'}, 'score_criterias': []}
------------------

วท.บ. สาขาวิชาเครื่องจักรกลและเมคคาทรอนิกส์เกษตร (ภาษาไทย ปกติ)
31
{'admission_criteria': {'admission_project': 31, 'faculty': 16, 'version': 2, 'is_deleted': False, 'created_at': '2021-06-04T16:59:35.267', 'created_by': 'cond16', 'additional_description': '', 'additional_condition': '', 'curriculum_majors_json': '[{"curriculum_major_id": 2056, "slots": 16, "major_cupt_code_id": 2, "program_code": "10020219500101A", "program_type_code": "A", "major_code": ""}]'}, 'score_criterias': [{'admission_criteria': 8005, 'version': 2, 'primary_order': 1, 'sec

{'admission_criteria': {'admission_project': 29, 'faculty': 28, 'version': 1, 'is_deleted': False, 'created_at': '2021-06-04T13:05:10.687', 'created_by': 'cond28', 'additional_description': '', 'additional_condition': '', 'curriculum_majors_json': '[{"curriculum_major_id": 82, "slots": 100, "major_cupt_code_id": 76, "program_code": "10020431610601A", "program_type_code": "A", "major_code": ""}]'}, 'score_criterias': []}
30
{'admission_criteria': {'admission_project': 30, 'faculty': 28, 'version': 1, 'is_deleted': False, 'created_at': '2021-06-04T13:12:25.441', 'created_by': 'cond28', 'additional_description': '', 'additional_condition': '', 'curriculum_majors_json': '[{"curriculum_major_id": 2158, "slots": 5, "major_cupt_code_id": 76, "program_code": "10020431610601A", "program_type_code": "A", "major_code": ""}]'}, 'score_criterias': []}
------------------

บธ.บ. สาขาวิชาการจัดการโรงแรมและท่องเที่ยว (ภาษาไทย ปกติ)
29
{'admission_criteria': {'admission_project': 29, 'faculty': 28, 'ver

{'admission_criteria': {'admission_project': 31, 'faculty': 6, 'version': 5, 'is_deleted': False, 'created_at': '2021-06-04T16:00:16.246', 'created_by': 'cond6', 'additional_description': '', 'additional_condition': '', 'curriculum_majors_json': '[{"curriculum_major_id": 2062, "slots": 4, "major_cupt_code_id": 167, "program_code": "10020105303501D", "program_type_code": "D", "major_code": ""}]'}, 'score_criterias': [{'admission_criteria': 7997, 'version': 5, 'primary_order': 1, 'secondary_order': 0, 'criteria_type': 'required', 'score_type': 'OTHER', 'value': None, 'unit': '', 'description': 'กำลังศึกษาหรือสำเร็จการศึกษาชั้นมัธยมศึกษาปีที่ 6 หรือกำลังศึกษาในชั้นปีสุดท้าย หรือ สำเร็จการศึกษาระดับมัธยมศึกษาตอนปลายจากต่างประเทศ หรือมีการเทียบวุฒิการศึกษา แบบ GED หรือเทียบเท่า ', 'parent': None, 'relation': None, 'pk': 70216}, {'admission_criteria': 7997, 'version': 5, 'primary_order': 1, 'secondary_order': 0, 'criteria_type': 'scoring', 'score_type': 'OTHER', 'value': '0.0000', 'unit': ''

In [8]:
final_data_to_json = {f'{p}-{m}': criterias[(p,m)]['data'] for p,m in criterias.keys()}

In [9]:
filename = 'export-to-portal-r4-all-updated.json'

with open(filename,'w') as f:
    print(json.dumps(final_data_to_json, indent=1),
          file=f)

In [10]:
print(json.dumps(final_data_to_json, indent=1))

{
 "10020101111801A-": {
  "major_cupt_code": {
   "program_code": "10020101111801A",
   "program_type": "\u0e20\u0e32\u0e29\u0e32\u0e44\u0e17\u0e22 \u0e1b\u0e01\u0e15\u0e34",
   "program_type_code": "A",
   "faculty": 2,
   "major_code": "",
   "title": "\u0e27\u0e17.\u0e1a. \u0e2a\u0e32\u0e02\u0e32\u0e27\u0e34\u0e0a\u0e32\u0e2d\u0e32\u0e2b\u0e32\u0e23 \u0e42\u0e20\u0e0a\u0e19\u0e32\u0e01\u0e32\u0e23 \u0e41\u0e25\u0e30\u0e01\u0e32\u0e23\u0e01\u0e33\u0e2b\u0e19\u0e14\u0e2d\u0e32\u0e2b\u0e32\u0e23",
   "major_title": "",
   "component_weight_type": "11"
  },
  "criterias": {}
 },
 "10020101500101A-": {
  "major_cupt_code": {
   "program_code": "10020101500101A",
   "program_type": "\u0e20\u0e32\u0e29\u0e32\u0e44\u0e17\u0e22 \u0e1b\u0e01\u0e15\u0e34",
   "program_type_code": "A",
   "faculty": 2,
   "major_code": "",
   "title": "\u0e27\u0e17.\u0e1a. \u0e2a\u0e32\u0e02\u0e32\u0e27\u0e34\u0e0a\u0e32\u0e40\u0e17\u0e04\u0e42\u0e19\u0e42\u0e25\u0e22\u0e35\u0e23\u0e30\u0e1a\u0e1a\u0e40\u0e01\